# Survival Multiarmed Bandits

## Comparing Multiple Algorithms using Repetitions

In [3]:
using_smpymab = False
using_gdrive = False

#pipy packages
if (using_smpymab):
  #install
  !pip install -q SMPyBandits
  #import
  #from SMPyBandits.Arms import Bernoulli #, Gaussian, Constant
  #from SMPyBandits.Policies import Uniform, EmpiricalMeans, UCB, UCBalpha, UCBV, klUCB, Thompson, SoftMix, BayesUCB

#link to google drive for importing .py files
if (using_gdrive):
  #access drive
  from google.colab import drive
  drive.mount('/content/drive')
  localpath = '/content/drive/My Drive/Colab Notebooks/MultiArmedBandits/MyMAB/'
  #local packages
  from importlib.machinery import SourceFileLoader
  #mabalgs = SourceFileLoader('mabalgs', localpath + 'mabalgs.py').load_module()
  #mabsim = SourceFileLoader('mabsim', localpath +  'mabsim.py').load_module()
  mabplot = SourceFileLoader('mabplot', localpath + 'mabplot.py').load_module()
  #import
  #from mabarms import ExtendedBernoulli
  #from mabalgs import SafeEpsilonGreedy, SafeUCB, SafeUCBalpha, ClassicEpsilonGreedy, ClassicEpsilonDecreasing, ClassicOptimisticGreedy, SafeKLUCB, PositiveGamblerUCB, GamblerBayesUCB, MaRaB, BanditGambler
  #from mabsim import mabs
  from mabplot import mabplt

#Dependencies
import numpy as np
from numpy.random import binomial, randint, uniform, choice, rand
from math import sqrt, log
from scipy.stats import beta
#from scipy.integrate import quad as integral
#from scipy.integrate import fixed_quad as integral
from scipy.integrate import quadrature as integral
import pandas as pd
from numba import jit
from tqdm import tqdm_notebook as tqdm
from collections import Iterable
#from IPython.display import display
import matplotlib.pyplot as plt
#import matplotlib.mlab as mlab
import datetime
%matplotlib inline
#%matplotlib notebook
#import pickle
#from google.colab import files

#current date
date = datetime.datetime.now().strftime("%Y_%m_%d")

In [4]:
""" partially copied from SMPyBandits"""

class Domain():

    def __init__(self, r_min=0.0, r_max=1.0):
        """ class for reward domain. """
        assert r_max >= r_min, "Error, the maximal reward must be greater than the minimal."  # DEBUG
        self.r_min = r_min  #: Lower values for rewards
        self.r_max = r_max  #: Higher values for rewards
        self.r_amp = r_max - r_min  #: Larger values for rewards
        self.r_0_1 = ((self.r_max==1.0) and (self.r_min==0.0))

################################################################################

class RandomArm():
    """ Base class for an arm class.
        return uniformly distributed random values between 0 and 1
    """

    def __init__(self):
        """ Base class for an arm class."""
        self.mean = 0.5

    def draw(self, shape=None):
        """ Draw a numpy array of random samples, of a certain shape. If shape is None, return a single sample"""
        return uniform(low=0.0, high=1.0, size=shape)

################################################################################

class BernoulliArm(RandomArm):
    """ Bernoulli distributed arm."""

    def __init__(self, p):
        """New arm."""
        super()
        assert 0.0 <= p <= 1.0, "Error, the parameter probability for Bernoulli class has to be in [0, 1]."  # DEBUG
        self.p = p  #: Parameter p for this Bernoulli arm
        self.mean = p

    # --- Random samples
    def draw(self, shape=None):
        """ Draw a numpy array of random samples, of a certain shape. If shape is None, return a single sample"""
        return binomial(1, self.p, size=shape)

################################################################################

class RandomPolicy():
    """ Base class for any policy.
        by default, choose an arm uniformly at random
    """

    def __init__(self, k, force_first_trial=True):
        """ New policy."""
        # Parameters
        assert k > 0, "Error, the number of arms must be a positive integer."  # DEBUG
        self.k = k  #: Number of Arms
        self.force_first_trial = force_first_trial   #: if True, each arm must be played at least once on the beginning 
        # Internal state
        self.t = 0  #: Internal time-step
        self.n_i = np.zeros(self.k, dtype=int)  #: Number of pulls of each arm
        self.i_last = 0
        self.bests = np.arange(k) #list of best arms (with higher utility)

    def reset(self):
        """ Start the game (fill pulls with 0)."""
        self.t = 0
        self.n_i.fill(0)
        self.i_last = 0
        self.bests = np.arange(k)

    def choose(self):
        r""" choose an arm with maximal index (uniformly at random):
        .. math:: A(t) \sim U(\arg\max_{1 \leq k \leq K} I_k(t)).
        .. note:: In almost all cases, there is a unique arm with maximal index, so we loose a lot of time with this generic code, but I couldn't find a way to be more efficient without loosing generality.
        """
        # Uniform choice among the best arms
        #print(self.bests, self.t, self.k)
        self.i_last = choice(self.bests)
        return self.i_last

    def observe(self, r):
        """ Receive reward, increase t, pulls, and update."""
        #update internal state
        self._update(r)
        #evaluate
        self._evaluate()
        #define bests
        if ( (self.force_first_trial) and (self.t < self.k) ):
          self.bests = np.flatnonzero(self.n_i == 0)
        else:
          self.bests = self._calc_bests()

    def _update(self, r):
        self.t += 1
        self.n_i[self.i_last] += 1

    def _evaluate(self):
        """ update utility after last observation """
        pass

    def _calc_bests(self):
        """ return list of best arms 
            by default, is just an uniform random arm
        """
        return np.array([randint(self.k)])

################################################################################

class EmpiricalMeansPolicy(RandomPolicy):
    """ Class that implements a generic index policy.
        by default, implements the empirical means method
        The naive Empirical Means policy for bounded bandits: like UCB but without a bias correction term. Note that it is equal to UCBalpha with alpha=0, only quicker.
    """

    def __init__(self, k, v_ini=None, force_first_trial=True):
        """ New generic index policy. """
        super().__init__(k, force_first_trial=force_first_trial)
        #self.mu_i = np.full(k, 0.0)  #: estimated mean for each arm
        self.s_i = np.full(k, 0.0)  #: cumulated rewards for each arm
        self.v_ini = v_ini  if  (v_ini is not None)  else  0.0   #: initial value (index or utility) for the arms
        self.v_i = np.full(k, v_ini)  #: value (index or utility) for each arm

    def reset(self):
        """ Initialize the policy for a new game."""
        super().reset()
        #self.mu_i.fill(0.0)
        self.s_i.fill(0.0)
        self.v_i.fill(self.v_ini)

    def _update(self, r):
        """ update estimated means after last observation """
        super()._update(r)
        self.s_i[self.i_last] += r
        #mu = self.mu_i[self.i_last]
        #n = self.n_i[self.i_last]
        #self.mu_i[self.i_last] = (mu * ((n-1) / n)) + (r / n)
        #biased_means = self.rewards / (1 + self.pulls)
        #estimated_means = self.rewards / np.maximum(1, self.pulls)

    def _evaluate(self):
        """ update utility after last observation 
            in this case, the utility is the estimated mean
        """
        self.v_i[self.i_last] = self.s_i[self.i_last] / self.n_i[self.i_last]

    def _calc_bests(self):
        """ define best arms - all with equivalent highest utility """
        return np.flatnonzero(self.v_i == np.max(self.v_i))

################################################################################

class EpsilonGreedyPolicy(EmpiricalMeansPolicy):
    r""" The epsilon-greedy random policy.
    - At every time step, a fully uniform random exploration has probability :math:`\varepsilon(t)` to happen, otherwise an exploitation is done.
    """

    def __init__(self, k, v_ini=None, force_first_trial=True, eps=0.9):
        super().__init__(k, v_ini=v_ini, force_first_trial=force_first_trial)
        assert 0 <= eps <= 1, "Error: the 'epsilon' parameter for EpsilonGreedy class has to be in [0, 1]."  # DEBUG
        self.eps = eps

    def _calc_bests(self):
        # Generate random number
        p = rand()
        """With a probability of epsilon, explore (uniform choice), otherwise exploit based on empirical mean rewards."""
        if p < self.eps: # Proba epsilon : explore
            return np.array([randint(0, self.k)])
        else:  # Proba 1 - epsilon : exploit
            return super()._calc_bests()

################################################################################

class UCBPolicy(EmpiricalMeansPolicy):

    def _evaluate(self):
        r""" Compute the current index, at time t and after :math:`N_k(t)` pulls of arm k:
        .. math:: I_k(t) = \frac{X_k(t)}{N_k(t)} + \sqrt{\frac{2 \log(t)}{N_k(t)}}.
        """
        #calculate utility following UCB formula
        i = self.i_last
        n_i = self.n_i[i]
        mu_i = self.s_i[i] / n_i
        t = self.t
        if self.n_i[i] == 0:
            self.v_i[i] = float('+inf')
        else:
            self.v_i[i] = mu_i + sqrt((2 * log(t)) / n_i)


################################################################################

class BernKLUCBPolicy(EmpiricalMeansPolicy):

    @jit
    def _klBern(self, x, y):
        r""" Kullback-Leibler divergence for Bernoulli distributions.
        .. math:: \mathrm{KL}(\mathcal{B}(x), \mathcal{B}(y)) = x \log(\frac{x}{y}) + (1-x) \log(\frac{1-x}{1-y}).
        """
        eps = 1e-15  #: Threshold value: everything in [0, 1] is truncated to [eps, 1 - eps]
        x = min(max(x, eps), 1 - eps)
        y = min(max(y, eps), 1 - eps)
        return x * log(x / y) + (1 - x) * log((1 - x) / (1 - y))

    @jit
    def _klucbBern(self, x, d, precision=1e-6):
        """ KL-UCB index computation for Bernoulli distributions, using :func:`klucb`."""
        upperbound = min(1., self._klucbGauss(x, d, sig2x=0.25))  # variance 1/4 for [0,1] bounded distributions
        return self._klucb(x, d, upperbound, precision)

    @jit
    def _klucbGauss(self, x, d, sig2x=0.25):
        """ KL-UCB index computation for Gaussian distributions.
        - Note that it does not require any search.
        .. warning:: it works only if the good variance constant is given.
        .. warning:: Using :class:`Policies.klUCB` (and variants) with :func:`klucbGauss` is equivalent to use :class:`Policies.UCB`, so prefer the simpler version.
        """
        return x + sqrt(abs(2 * sig2x * d))

    @jit
    def _klucb(self, x, d, upperbound, precision=1e-6, lowerbound=float('-inf'), max_iterations=50):
        r""" The generic KL-UCB index computation.
        - ``x``: value of the cum reward,
        - ``d``: upper bound on the divergence,
        - ``kl``: the KL divergence to be used (:func:`klBern`, :func:`klGauss`, etc),
        - ``upperbound``, ``lowerbound=float('-inf')``: the known bound of the values ``x``,
        - ``precision=1e-6``: the threshold from where to stop the research,
        - ``max_iterations=50``: max number of iterations of the loop (safer to bound it to reduce time complexity).
        .. math::
            \mathrm{klucb}(x, d) \simeq \sup_{\mathrm{lowerbound} \leq y \leq \mathrm{upperbound}} \{ y : \mathrm{kl}(x, y) < d \}.
        .. note:: It uses a **bisection search**, and one call to ``kl`` for each step of the bisection search.
        For example, for :func:`klucbBern`, the two steps are to first compute an upperbound (as precise as possible) and the compute the kl-UCB index:
        >>> x, d = 0.9, 0.2   # mean x, exploration term d
        >>> upperbound = min(1., klucbGauss(x, d, sig2x=0.25))  # variance 1/4 for [0,1] bounded distributions
        """
        v = max(x, lowerbound)
        u = upperbound
        i = 0
        while ((i < max_iterations) and (u - v > precision)):
            i += 1
            m = (v + u) * 0.5
            if self._klBern(x, m) > d:
                u = m
            else:
                v = m
        return (v + u) * 0.5

    def _evaluate(self):
        r""" Compute the current index, at time t and after :math:`N_k(t)` pulls of arm k:
        .. math::
            \hat{\mu}_k(t) &= \frac{X_k(t)}{N_k(t)}, \\
            U_k(t) &= \sup\limits_{q \in [a, b]} \left\{ q : \mathrm{kl}(\hat{\mu}_k(t), q) \leq \frac{c \log(t)}{N_k(t)} \right\},\\
            I_k(t) &= U_k(t).
        If rewards are in :math:`[a, b]` (default to :math:`[0, 1]`) and :math:`\mathrm{kl}(x, y)` is the Kullback-Leibler divergence between two distributions of means x and y (see :mod:`Arms.kullback`),
        and c is the parameter (default to 1).
        """
        c = 1.0
        #tolerance = 1e-4
        i = self.i_last
        n_i = self.n_i[i]
        mu_i = self.s_i[i] / n_i
        t = self.t
        if n_i == 0:
            self.v_i[i] = float('+inf')
        else:
            self.v_i[i] = self._klucbBern(mu_i, c * log(t) / n_i)

################################################################################

class ThompsonPolicy(EmpiricalMeansPolicy):
    r"""The Thompson (Bayesian) index policy.
    - By default, it uses a Beta posterior (:class:`Policies.Posterior.Beta`), one by arm.
    - Prior is initially flat, i.e., :math:`a=\alpha_0=1` and :math:`b=\beta_0=1`.
    - A non-flat prior for each arm can be given with parameters ``a`` and ``b``, for instance::
        nbArms = 2
        prior_failures  = a = 100
        prior_successes = b = 50
        policy = Thompson(nbArms, a=a, b=b)
        np.mean([policy.choice() for _ in range(1000)])  # 0.515 ~= 0.5: each arm has same prior!
    - A different prior for each arm can be given with parameters ``params_for_each_posterior``, for instance::
        nbArms = 2
        params0 = { 'a': 10, 'b': 5}  # mean 1/3
        params1 = { 'a': 5, 'b': 10}  # mean 2/3
        params = [params0, params1]
        policy = Thompson(nbArms, params_for_each_posterior=params)
        np.mean([policy.choice() for _ in range(1000)])  # 0.9719 ~= 1: arm 1 is better than arm 0 !
    - Reference: [Thompson - Biometrika, 1933].
    """

    def _evaluate(self):
        r""" Compute the current index, at time t and after :math:`N_k(t)` pulls of arm k, giving :math:`S_k(t)` rewards of 1, by sampling from the Beta posterior:
        .. math::
            A(t) &\sim U(\arg\max_{1 \leq k \leq K} I_k(t)),\\
            I_k(t) &\sim \mathrm{Beta}(1 + \tilde{S_k}(t), 1 + \tilde{N_k}(t) - \tilde{S_k}(t)).
        """
        for i in range(self.k):
          a = self.s_i[i] + 1
          b = self.t - self.s_i[i] + 1
          self.v_i[i] = beta.rvs(a, b)


################################################################################

class BayesUCBPolicy(EmpiricalMeansPolicy):
    """ The Bayes-UCB policy.
    - By default, it uses a Beta posterior (:class:`Policies.Posterior.Beta`), one by arm.
    -Reference: [Kaufmann, Cappé & Garivier - AISTATS, 2012].
    """
    def _evaluate(self):
        r""" Compute the current index, at time t and after :math:`N_k(t)` pulls of arm k, giving :math:`S_k(t)` rewards of 1, by taking the :math:`1 - \frac{1}{t}` quantile from the Beta posterior:
        .. math:: I_k(t) = \mathrm{Quantile}\left(\mathrm{Beta}(1 + S_k(t), 1 + N_k(t) - S_k(t)), 1 - \frac{1}{t}\right).
        """
        i = self.i_last
        q = 1. - 1. / (1 + self.t)
        a = self.s_i[i] + 1
        b = self.t - self.s_i[i] + 1
        self.v_i[i] = beta.ppf(q, a, b)

################################################################################

# class for the marab algorithm
class MaRaBPolicy(UCBPolicy):
    
    def __init__(self, k, v_ini=None, force_first_trial=True, alpha=0.05, C=1e-6):
        super().__init__(k, v_ini=v_ini, force_first_trial=force_first_trial)
        self.alpha = alpha
        self.C = C
        self.reward_samples = [np.array([0.0]) for a in range(k)]
        
    def reset(self):
        super().reset()
        self.reward_samples = [np.array([0.0]) for a in range(self.k)]
                                       
    def _update(self, r):
        super()._update(r)
        self.reward_samples[self.i_last] = np.sort(np.append(self.reward_samples[self.i_last], [r]))
        
    def _evaluate(self):
        i = self.i_last
        # calculating empirical cvar
        e = np.ceil(self.alpha*self.n_i[i]).astype(int)
        empirical_cvar = self.reward_samples[i][:e].mean()
        # calculating lower confidence bound
        lcb = np.sqrt(np.log(np.ceil(self.alpha*self.t))/self.n_i[i])
        # adding score to scores list
        self.v_i[i] = empirical_cvar - self.C * lcb

################################################################################

class Budgeted:

    def __init__(self, k, d=None, b_0=None):
        if b_0 is None:
          b_0=k
        self.b_0 = b_0
        self.d = d  if  (isinstance(d, Domain))  else  Domain()
        self.b = b_0   #budget
        self.s = 0.0   #total cumulated rewards
        
    def reset(self):
        self.b = self.b_0
        self.s = 0.0

    def _update(self, r):
        self.s += r
        self.b += r * self.d.r_amp + self.d.r_min

################################################################################

class BanditGamblerPolicy(EmpiricalMeansPolicy, Budgeted):

    def __init__(self, k, d=None, b_0=None):
        EmpiricalMeansPolicy.__init__(self, k)
        Budgeted.__init__(self, k, d=d, b_0=b_0)

    #@jit
    def ruin_estimated_prob(self, i):
        n_i = self.n_i[i]
        x_i = self.s_i[i]
        y_i = n_i - self.s_i[i]
        b = max(1.0, self.b)
        return beta.cdf(0.5, x_i+1, y_i+1) + integral(lambda p, x, y, b : ((1-p)/p)**b * beta.pdf(p, x+1, y+1), 0.5, 1.0, (x_i, y_i, b))[0]

    def reset(self):
        super().reset()
        #IndexPolicy.startGame(self)
        #Budgeted.startGame(self)
        #BernoulliEstimator.startGame(self)

    def _update(self, r):
        super()._update(r)
        #IndexPolicy.getReward(self, arm, reward)
        #Budgeted.getReward(self, reward)
        #BernoulliEstimator.getReward(self, arm, reward)

    def _evaluate(self):
        i = self.i_last
        self.v_i[i] = 1.0 - self.ruin_estimated_prob(i)

################################################################################

class MAB():
    """ Base MAB process. """

    def __init__(self, A, G, h, d=None, n=1, w=None, b_0=None, run=False, save_only_means=True):
        """
         A : List of Arms
         G : List of Algorithms
         h : max time-horizon
         d : rewards domain
         n : number of repetitions
         w : sliding window
         b_0 : initial budget
        """
        #domain of rewards ( by default on [0, 1] )
        self.d = d  if  isinstance(d, Domain)  else  Domain()

        #time-horizon (0, 1 ... t ... h)
        self.h = h   #time-horizon
        self.T = range(self.h)          #range for time (0 ... h-1)
        self.T1 = range(1, self.h+1)    #range for time (1 ... h)
        self.T01 = range(0, self.h+1)   #range for time (0, 1 ... h)

        #arms (1 ... i ... k)
        self.A = A if isinstance(A, Iterable) else [A]

        #number of arms
        self.k = len(self.A)
        self.K = range(self.k)          #range for arms (0 ... k-1)
        self.K1 = range(1,self.k+1)     #range for arms (1 ... k)

        #arms properties
        self.mu_a = np.array([a.mean for a in A])  #means
        self.mu_star = np.max(self.mu_a)           #best mean
        self.a_star = np.argmax(self.mu_a)         #best arm index
        self.mu_worst = np.min(self.mu_a)          #worst mean
        self.a_worst = np.argmin(self.mu_a)        #worst arm index

        #budget
        self.b_0 = b_0   if  (b_0 is not None)  else   k
        
        #algorithms (1 ... g ... m)
        self.G = G if isinstance(G, Iterable) else [G]
        self.m = len(self.G)

        #repetitions (1 ... j ... n)
        self.n = n

        #window
        if (w is not None):
            self.w = max(2, min(w, horizon-1))
        else:
            self.w = w

        #if save all sim data
        self.save_only_means = save_only_means		

        #run
        if run:
            self.run()


    def run(self, tqdm_desc_it="iterations", tqdm_desc_alg="algorithms", tqdm_desc_rep="repetitions", tqdm_leave=False, tqdm_disable=False, prev_draw=True):

        #time-horizon (1 ... t ... h)
        #arms (1 ... i ... k)
        #repetitions (1 ... j ... n)
        #algorithms (1 ... g ... m)

        # Initialize Rewards and History of selected Actions (3d matrices [t x g x i])
        X = np.zeros((self.n, self.m, self.h), dtype=float)
        R = np.zeros((self.n, self.m, self.h), dtype=float)
        H = np.zeros((self.n, self.m, self.h), dtype=int)

        # Draw for every arm all repetitions
        if prev_draw:
            X_i_t_j = np.array([arm.draw((self.h, self.n)) for arm in self.A])	

        # For each repetition
        #for j in tqdm(range(self.n), desc=tqdm_desc_rep, leave=(tqdm_leave and self.m == 1), disable=(tqdm_disable or self.n == 1)):
        for j in tqdm(range(self.n), desc=tqdm_desc_rep, leave=tqdm_leave, disable=(tqdm_disable or self.n == 1)):

            # For each algorithm
            #for g, alg in enumerate(tqdm(self.G, desc=tqdm_desc_alg, leave=tqdm_leave, disable=(tqdm_disable or self.m == 1))):
            for g, alg in enumerate(self.G):

                # Initialize
                alg.reset()

                # Loop on time
                for t in tqdm(self.T, desc=tqdm_desc_it, leave=tqdm_leave, disable=(tqdm_disable or self.n > 1 or self.m > 1) ):
                    # The algorithm chooses the arm to play
                    i = alg.choose()
                    # The arm played gives reward
                    if prev_draw:
                        x = X_i_t_j[i, t, j]
                    else:
                        x = self.A[i].draw()
                    # The reward is returned to the algorithm
                    alg.observe(x)
                    # Save both
                    X[j, g, t] = x
                    H[j, g, t] = i

        #extended rewards domain
        R = X * self.d.r_amp + self.d.r_min
        
        #actions history, with initial action index being 1, not 0
        H1 = H+1

        #actions map (bool 4d matrix)
        H_a = np.array([[[[True if (H[j,g,t]==i) else False for t in self.T] for i in self.K] for g in range(self.m)] for j in range(self.n)], dtype='bool')

        #progressive actions count (int 4d matrix [t x j x i x a])
        N_a = np.cumsum(H_a, axis=3)

        #averaged progressive actions count (float 3d matrix [t x j x a]) #averaged over repetitions
        self.MN_a = np.mean(N_a, axis=0)		

        #progressive actions frequency (float 4d matrix [t x j x i x a])
        F_a = N_a / self.T1

        #averaged progressive actions frequency (float 3d matrix [t x j x a]) #averaged over repetitions
        self.MF_a = np.mean(F_a, axis=0)

        if (self.w is not None):

            #window count (int 4d matrix [t x j x i x a])
            NW_a = np.concatenate((N_a[:,:,:,:self.w], N_a[:,:,:,self.w:] - N_a[:,:,:,:-self.w]), axis=3)

            #averaged window count (float 3d matrix [t x j x a]) #averaged over repetitions
            self.MNW_a = np.mean(NW_a, axis=0)		

            #window frequency (float 4d matrix [t x j x i x a])
            FW_a = np.concatenate((N_a[:,:,:,:self.w] / np.arange(1,self.w+1, dtype='float'), (N_a[:,:,:,self.w:] - N_a[:,:,:,:-self.w]) / float(self.w)), axis=3) 

            #averaged window frequency (float 3d matrix [t x j x a]) #averaged over repetitions
            self.MFW_a = np.mean(FW_a, axis=0)		

        #final arm pull count (int 3d matrix [j x i x a])
        n_a = N_a[:,:,:,self.h-1]

        #averaged final arm pull count (float 2d matrix [j x a]) #averaged over repetitions
        self.mn_a = np.mean(n_a, axis=0)

        #final arm pull frequency (float 3d matrix [j x i x a])
        f_a = F_a[:,:,:,self.h-1]

        #averaged final arm pull frequency (float 2d matrix [j x a]) #averaged over repetitions
        self.mf_a = np.mean(f_a, axis=0)

        #progressive cumulative rewards (float 3d matrix [t x j x i])
        SR = np.cumsum(R, axis=2, dtype='float')

        #averaged progressive cumulative rewards (float 2d matrix [t x j]) #averaged over repetitions
        self.MSR = np.mean(SR, axis=0)

        #final rewards (float 2d matrix [j x i])
        sr = SR[:,:,self.h-1]

        #averaged final rewards (float 1d matrix [j]) #averaged over repetitions
        self.msr = np.mean(sr, axis=0)
        #and standard deviation
        self.dsr = np.std(sr, axis=0)

        #progressive average rewards (float 3d matrix [t x j x i]) #averaged over time
        MR = SR / self.T1

        #averaged progressive average rewards (float 2d matrix [t x j]) #averaged over time and repetitions
        self.MMR = np.mean(MR, axis=0)

        #regret (float 3d matrix [t x j x i])
        L = self.mu_star - R

        #averaged regret (float 2d matrix [t x j])
        #self.ML = np.mean(L, axis=0)
        #progressive average regret (float 3d matrix [t x j x i]) #averaged over time
        ML = self.mu_star - MR

        #averaged average regret (float 2d matrix [t x j]) #averaged over time and repetitions
        self.MML = np.mean(ML, axis=0)

        #cumulated regret (float 3d matrix [t x j x i])
        SL = np.cumsum(L, axis=2, dtype='float')

        #averaged cumulated regret (float 2d matrix [t x j]) #averaged over repetitions
        self.MSL = np.mean(SL, axis=0)

        #final cumulated regret (float 2d matrix [j x i])
        sl = SL[:,:,self.h-1]

        #averaged final cumulated regret (float 1d matrix [j]) #averaged over repetitions
        self.msl = np.mean(sl, axis=0)
        #and standard deviation
        self.dsl = np.std(sl, axis=0)
        
        #rewards map (float 4d matrix [t x j x i x a])
        R_a = np.array([[[[R[j,g,t] if (H[j,g,t]==i) else 0.0 for t in self.T] for i in self.K] for g in range(self.m)] for j in range(self.n)], dtype='float')

        #averaged rewards map (float 3d matrix [t x j x a]) #averaged over repetitions
        self.MR_a = np.mean(R_a, axis=0)

        #progressive rewards map (int 4d matrix [t x j x i x a])
        SR_a = np.cumsum(R_a, axis=3)

        #averaged progressive rewards map (float 3d matrix [t x j x a]) #averaged over repetitions
        self.MSR_a = np.mean(SR_a, axis=0)

        #final rewards per action (float 3d matrix [j x i x a])
        sr_a = SR_a[:,:,:,self.h-1]

        #averaged final rewards per action (float 2d matrix [j x a]) #averaged over repetitions
        self.msr_a = np.mean(sr_a, axis=0)

        #reward proportion per action (float 3d matrix [j x i x a])
        fr_a = sr_a / SR[:,:,self.h-1,np.newaxis]

        #averaged proportion per action (float 2d matrix [j x a]) #averaged over repetitions
        self.mfr_a = np.mean(fr_a, axis=0)

        #progressive budget (float 3d matrix [t x j x i])
        # i.e. the progressive cumulative rewards plus initial budget
        B = SR + self.b_0

        ##progressive on negative counter of episodes (float 3d matrix [t x j])
        ## i.e. the number of episodes where, at each time t, alg j is running on negative budget
        #N = np.sum(B >= 0, axis=0)

        #averaged progressive budget (float 2d matrix [t x j]) #averaged over repetitions
        #self.MB = np.mean(B, axis=0)
        self.MB = self.MSR + self.b_0

        #final budget (float 2d matrix [j x i])
        b = B[:,:,self.h-1]

        #averaged final budget (float 1d matrix [j]) #averaged over repetitions
        self.mb = np.mean(b, axis=0)

        #time map on negative budget (int 3d matrix [t x j x i])
        TNB = np.array([[[1 if(v<0) else 0 for v in B_ij] for B_ij in B_i] for B_i in B])
        
        #time dead map (int 3d matrix [t x j x i])
        TD = np.maximum.accumulate(TNB, axis=2)

        #progressive survival counter of episodes (float 3d matrix [t x j])
        self.SC = 1 - np.mean(TD, axis=0)
        #final survival counter
        self.sc = self.SC[:,self.h-1]
        #final survival rate
        self.rsc = self.sc / self.n
         
        ##time map of the averaged budget on negative (int 2d matrix [t x j])
        #self.TNMB = np.array([[1 if(v<0) else 0 for v in MB_j] for MB_j in self.MB])

        ##survival time (before ruin or end) (int 2d matrix [j x i])
        #Z = np.reshape(np.ones(self.n*self.m, dtype='int'), [self.n, self.m, 1]) #add 1 at the end		
        #TNBZ = np.block([TNB, Z])
        #self.TTNB = np.array([[np.nonzero(v_tj==1)[0][0] for v_tj in v_t] for v_t in TNBZ])		

        ##averaged survival time (before ruin or end) (int 1d matrix [j])
        #self.MTTNB = np.mean(self.TTNB, axis=0)
        ##and std dev
        #self.DTTNB = np.std(self.TTNB, axis=0)

        ##cumulated time progression on negative budget
        #STNB = np.cumsum(TNB, axis=2)
        #self.STNMB = np.cumsum(self.TNMB, axis=1) 
        ##self.MSTNB = np.mean(self.STNB, axis=0)
        #
        ##final cumulated time on negative budget
        #stnb = STNB[:,:,self.tau-1]
        #
        #self.stnmb = self.STNMB[:,self.tau-1]
        #
        ##averaged final cumulated time on negative budget
        #self.mstnb = np.mean(stnb, axis=0)
        ##and std dev
        #self.dstnb = np.std(stnb, axis=0)

        ##ruin episodes (int 1d matrix [j])
        #self.senb = np.count_nonzero(stnb, axis=0) 
        ##rate
        #self.renb = 1.0 - self.senb / self.n

        ##negative budget progression
        #NB = np.array([[[v if(v<0) else 0 for v in B_ij] for B_ij in B_i] for B_i in B])
        #
        ##average negative budget progression
        #self.NMB = np.array([[v if(v<0) else 0 for v in MB_j] for MB_j in self.MB])
        #
        ##cumulated negative budget progression
        #SNB = np.cumsum(NB, axis=2, dtype='float')
        #
        ##self.MSNB = np.mean(SNB, axis=0)
        #
        ##cumulated negative budget progression on average
        #self.SNMB = np.cumsum(self.NMB, axis=1, dtype='float') 
        #
        ##final cumulated negative budget
        #snb = SNB[:,:,self.tau-1]
        #
        #self.snmb = self.SNMB[:,self.tau-1]
        #
        ##final cumulated negative budget (float 1d matrix [j]) #averaged over repetitions
        #self.msnb = np.mean(snb, axis=0)
        ##and its std deviation
        #self.dsnb = np.std(snb, axis=0)

        if(not self.save_only_means):
            self.R = R
            self.H = H
            self.H1 = H1
            self.H_a = H_a
            self.R_a = R_a
            self.N_a = N_a
            self.F_a = F_a
            self.n_a = n_a
            self.f_a = f_a
            self.NW_a = NW_a
            self.SR = SR
            self.sr = sr
            self.MR = MR
            self.L = L
            self.ML = ML
            self.SL = SL
            self.B = B
            self.b = b
            self.TNB = TNB
            self.STNB = STNB
            self.NB = NB
            self.SNB = SNB
            self.snb = snb



## Setting (10-Bernoulli-Arms)

In [5]:
plt.rcParams['figure.figsize'] = (16, 8)

k = 10
means = np.linspace(0.0, 0.6, k)

#BERNOULLI
A = [BernoulliArm(m) for m in means]

#initial budget
#b_0 = 20.0
b_0 = k
b_s = 3.0

#domain support for rewards
d = Domain(r_min=-1.0, r_max=+1.0)

#algorithms
G = [
     EmpiricalMeansPolicy(k),
     EpsilonGreedyPolicy(k, eps=0.1), 
     ##SafeEpsilonGreedy(k, epsilon=0.1, inibudget=b_0, safebudget=b_s, lower=lower, amplitude=ampl),
     #SoftMix(k, lower=lower, amplitude=ampl), ##implementation to be verified...
     #SafeKLUCB(k, inibudget=b_0, safebudget=b_s, lower=lower, amplitude=ampl),
     BernKLUCBPolicy(k),
     ##SafeUCBalpha(k, alpha=1.0*ampl, inibudget=b_0, safebudget=b_s, lower=lower, amplitude=ampl),
     ##UCBalpha(k, alpha=1.0*ampl, lower=minr, amplitude=ampl),
     #SafeUCB(k, inibudget=b_0, safebudget=b_s, lower=lower, amplitude=ampl),
     UCBPolicy(k),
     #PositiveGamblerUCB(k, lower=lower, amplitude=ampl),
     MaRaBPolicy(k),
     ###UCBalpha(k, alpha=4.0*ampl, lower=minr, amplitude=ampl), 
     ###UCBalpha(k, alpha=0.5*ampl),
     #UCBV(k, lower=lower, amplitude=ampl),
     ThompsonPolicy(k),
     BayesUCBPolicy(k),
     BanditGamblerPolicy(k, d=d, b_0=b_0)  
    ]

crits = [
    "Survival Episodes Rate", 
    "Survival Time (before ruin or end) (averaged over episodes)", 
    "Survival Time (std dev)", 
    "Cumulative Time on Negative Budget (averaged over episodes)", 
    "Cumulative Time on Negative Budget (std dev)", 
    "Cumulative Negative Budget (averaged over episodes)",
    "Cumulative Negative Budget (std dev)",
    "Cumulative Regret (averaged over episodes)",
    "Cumulative Regret (std dev)"
    ]
labels = ["EmpiricalMeans", "$\epsilon$-Greedy", "KL-UCB", "UCB", "MaRaB", "Thompson", "BayesUCB", "BG"]
colors=['r', 'r', 'orange', 'g', 'g', 'b', 'b', 'm', 'c', 'tan', 'pink', 'k']
styles=['-', '--', '-', '--', '-', '--', '-', '-', ':', '-.', ':', '-']


## Simulation Bernoulli : short-horizon

 - Several Repetitions 
 - Short Horizon

In [6]:
#time-horizon
h = 200

#repetitions
n = 10

M1 = MAB(A, G, h, d=d, n=n, b_0=b_0)
M1.run(tqdm_leave=True)

<ipython-input-4-2c5b49fb1301>:480: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for j in tqdm(range(self.n), desc=tqdm_desc_rep, leave=tqdm_leave, disable=(tqdm_disable or self.n == 1)):


repetitions:   0%|          | 0/10 [00:00<?, ?it/s]

<ipython-input-4-2c5b49fb1301>:490: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for t in tqdm(self.T, desc=tqdm_desc_it, leave=tqdm_leave, disable=(tqdm_disable or self.n > 1 or self.m > 1) ):
<ipython-input-4-2c5b49fb1301>:204: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "_klucbBern" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-4-2c5b49fb1301> (207)

File "<ipython-input-4-2c5b49fb1301>", line 207:
    def _klucbBern(self, x, d, precision=1e-6):
        <source elided>
        """ KL-UCB index computation for Bernoulli distributions, using :func:`klucb`."""
        upperbound = min(1., self._klucbGauss(x, d, sig2x=0.25))  # variance 1/4 for [0,1] bounded distributions
        ^

  @jit
C:\Anaconda3\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "_klucbBe

In [7]:
P1 = mabplt(M1)

#values = [M1.rsc, M1.MTTNB, M1.DTTNB, M1.mstnb, M1.dstnb, M1.msnb, M1.dsnb, M1.msl, M1.dsl]
#
#df = pd.DataFrame(values, crits, columns=labels)
#display(df)

#P1.plot_comp_algs_ruined_episodes(names=labels, names_rotation='horizontal', ylabel="Survival Episodes", compact_view=False, title="") #, filename='images/surv_rate_B_t200.pdf')
#P1.plot_comp_algs_survival_time(names=labels, names_rotation='horizontal', compact_view=False, title="") # filename='images/surv_time_B_t200.pdf')
#P1.plot_comp_algs_cumulated_regret(names=labels, names_rotation='horizontal', compact_view=False, title="") #, filename='images/regret_B_t200.pdf')
#P1.plot_comp_algs_cumulated_negative_budget(names=labels, names_rotation='horizontal', compact_view=False, title="")

P1.plot_survival_progression(show=False, title="", ylabel="Survival Rate", names=labels, linestyles=styles, linecolors=colors)
plt.savefig(date + '_survival_progression_tau' + str(tau) + '.pdf', bbox_inches='tight')
files.download(date + '_survival_progression_tau' + str(tau) + '.pdf')
plt.show()

P1.plot_budget_progression(show=False, title="", ylabel="Budget (average)", names=labels, linestyles=styles, linecolors=colors)
plt.savefig(date + '_budget_progression_tau' + str(tau) + '.pdf', bbox_inches='tight')
files.download(date + '_budget_progression_tau' + str(tau) + '.pdf')
plt.show()

P1.plot_precision_progression(show=False, title="", ylabel="Precision (average)", names=labels, linestyles=styles, linecolors=colors)
plt.savefig(date + '_precision_progression_tau' + str(tau) + '.pdf', bbox_inches='tight')
files.download(date + '_precision_progression_tau' + str(tau) + '.pdf')
plt.show()

P1.plot_average_reward_progression(show=False, title="", names=labels, linestyles=styles, linecolors=colors)
plt.savefig(date + '_avg_reward_progression_tau' + str(tau) + '.pdf', bbox_inches='tight')
files.download(date + '_avg_reward_progression_tau' + str(tau) + '.pdf')
plt.show()

#P1.plot_cumulated_negative_budget_progression(title="", names=labels, linestyles=styles, linecolors=colors)

#P1.plot_negative_budget_progression(title="", names=labels, linestyles=styles, linecolors=colors)

#P1.plot_negative_budget_time_progression(title="", names=labels, linestyles=styles, linecolors=colors)

P1.plot_cumulated_regret_progression(show=False, title="", ylabel="Cumulated Regret (average)", names=labels, linestyles=styles, linecolors=colors)
plt.savefig(date + '_regret_progression_tau' + str(tau) + '.pdf', bbox_inches='tight')
files.download(date + '_regret_progression_tau' + str(tau) + '.pdf')
plt.show()

#for j, g in enumerate(M1.G):
#    P1.plot_survival_histogram(j=j, title=str(g))

NameError: name 'mabplt' is not defined

## Simulation Bernoulli : long-horizon

 - Few Repetitions 
 - Long Horizon

In [8]:
#time-horizon
tau = 2000 #15000 

#repetitions
n = 20 #100

M2 = MAB(A, G, tau, d=d, repetitions=n, window=win, inibudget=b_0)
M2.run(tqdm_leave=True)

NameError: name 'win' is not defined

In [0]:
P2 = mabplt(M2)

values = [M2.renb, M2.MTTNB, M2.DTTNB, M2.mstnb, M2.dstnb, M2.msnb, M2.dsnb, M2.msl, M2.dsl]

df = pd.DataFrame(values, crits, labels)
display(df)

#P2.plot_comp_algs_ruined_episodes(names=labels, names_rotation='horizontal', compact_view=False, title="")
#P2.plot_comp_algs_cumulated_negative_budget(names=labels, names_rotation='horizontal', compact_view=False, title="")
#P2.plot_comp_algs_total_rewards(names=labels, names_rotation='horizontal', compact_view=False, title="")
#P2.plot_comp_algs_cumulated_regret(names=labels, names_rotation='horizontal', compact_view=False, title="")

P2.plot_survival_progression(show=False, title="", ylabel="Survival Rate", names=labels, linestyles=styles, linecolors=colors)
plt.savefig(date + '_survival_progression_tau' + str(tau) + '.pdf', bbox_inches='tight')
files.download(date + '_survival_progression_tau' + str(tau) + '.pdf')
plt.show()

P2.plot_budget_progression(show=False, title="", ylabel="Budget (average)", names=labels, linestyles=styles, linecolors=colors)
#plt.ylim(-600, 1800)
plt.savefig(date + '_budget_progression_tau' + str(tau) + '.pdf', bbox_inches='tight')
files.download(date + '_budget_progression_tau' + str(tau) + '.pdf')
plt.show()

P2.plot_cumulated_regret_progression(show=False, title="", ylabel="Regret (average)", names=labels, linestyles=styles, linecolors=colors)
#plt.ylim(0, 1000)
plt.savefig(date + '_regret_progression_tau' + str(tau) + '.pdf', bbox_inches='tight')
files.download(date + '_regret_progression_tau' + str(tau) + '.pdf')
plt.show()

P2.plot_average_reward_progression(title="", names=labels, linestyles=styles, linecolors=colors)

P2.plot_precision_progression(title="", names=labels, linestyles=styles, linecolors=colors)

P2.plot_cumulated_negative_budget_progression(title="", names=labels, linestyles=styles, linecolors=colors)
P2.plot_negative_budget_progression(title="", names=labels, linestyles=styles, linecolors=colors)
P2.plot_negative_budget_time_progression(title="", names=labels, linestyles=styles, linecolors=colors)
P2.plot_cumulated_regret_progression(title="", ylabel="Cumulated Regret (average)", names=labels, linestyles=styles, linecolors=colors)
